# arXiv Paper Embedding

## On a Single GPU
This notebook utilizes 1 GPU.

In [ ]:
%%capture
!pip install sentence_transformers

In [ ]:
%%capture
!pip install semanticscholar

In [ ]:
import cudf
import pandas as pd
import json
import os
import re
from tqdm import tqdm
import string
import pickle
import multiprocessing
import src.scholar_citations as sch_script
import src.vectors as vect

DATA_PATH = "./arxiv-metadata-oai-snapshot.json"

In [ ]:
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

## Step 1: Data pre processing
Before we do anything else, we need to load the papers dataset, do some basic cleaning, and get it into a workable format. Below,
we will use CuDF to house the data and apply seom transformations in a generator, loading from file.

In [ ]:
def clean_description(description: str):
    if not description:
        return ""
    # remove unicode characters
    description = description.encode('ascii', 'ignore').decode()

    # remove punctuation
    description = re.sub('[%s]' % re.escape(string.punctuation), ' ', description)

    # clean up the spacing
    description = re.sub('\s{2,}', " ", description)

    # remove urls
    #description = re.sub("https*\S+", " ", description)

    # remove newlines
    description = description.replace("\n", " ")

    # remove all numbers
    #description = re.sub('\w*\d+\w*', '', description)

    # split on capitalized words
    description = " ".join(re.split('(?=[A-Z])', description))

    # clean up the spacing again
    description = re.sub('\s{2,}', " ", description)

    # make all words lowercase
    description = description.lower()

    return description

In [ ]:
data = vect.load_raw_data(DATA_PATH)
data = pd.DataFrame.from_records(data)

In [ ]:
data.categories = data.categories.str.lower()
condition = (
    (data.categories.str.contains('cs.lg')) |
    (data.categories.str.contains('cs.ai')) |
    (data.categories.str.contains('cs.cl')) |
    (data.categories.str.contains('cs.cv')) |
    (data.categories.str.contains('cs.ne')) 
)
data_ds = data[condition]

In [ ]:
def get_categories(cat_str: str):
    cat_str = [elem for elem in cat_str.split(" ") if elem]
    return ", ".join(cat_str)
data_ds['categories'] = [get_categories(x) for x in data_ds.categories]

In [ ]:
data_ds = sch_script.fill_year_column(data_ds)

In [ ]:
input_ = data_ds['title'] + data_ds['abstract']
data_ds['input'] =  input_.apply(lambda x: clean_description(x))

In [ ]:
cdf = cudf.DataFrame.from_pandas(data_ds)

In [ ]:
len(cdf)

In [ ]:
cdf.year.value_counts()

## Step 2: Create sentence embeddings
Here I use a cookie-cutter -- **out of the box** -- model from HuggingFace to transform papers abstracts + titles into vectors.

**This takes a long time**... So best to take a subset. Or use the dask cluster for multi-gpu encoding.

In [ ]:
cdf = cdf.reset_index(drop=True)

In [ ]:
# change if needed
sample = len(cdf)
batch = cdf[:sample]

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

vectors = model.encode(
    sentences = batch.input.values_host,
    normalize_embeddings = True,
    batch_size = 64,
    show_progress_bar = True
)

In [ ]:
# Vectors created!
batch['vector'] = cudf.Series(vectors.tolist(), index=batch.index)

In [ ]:
batch.to_json("ds_embeddings.json")

### Get papers citations using Semantic Scholar API requests : please add api_key in `src.scholar_citations`

In [ ]:
#ids = pd.read_json("./ds_embeddings.json").id
ids = batch.id.to_pandas()

In [ ]:
results = []
items = list(zip(
    ids,
    ["./citations.json" for i in range(len(ids))],
    )
)

In [ ]:
len(ids)

In [ ]:
with multiprocessing.Pool() as pool:
    for res in tqdm(pool.starmap(sch_script.get_sch_paper, items)):
        results.append(res)

### Clean citations and get missing/error due to deconnections : run this part few times until all ids are there

In [ ]:
files = [file for file in os.listdir('./') if ("citations_" in file) or ("missing_ids" in file)]
files

In [ ]:
data = []
messy = []

for file in files:
    with open(file, "r") as f:
        for jsonObj in f:
            try:
                dd = json.loads(jsonObj)
                data.append(dd)
            except:
                messy.append(jsonObj)

In [ ]:
data = pd.DataFrame.from_records(data)

In [ ]:
missing_ids = set(emb.id) - set(data.arxiv_id)
len(missing_ids)

In [ ]:
results_missing = []
items = list(zip(
    missing_ids,
    ["./missing_ids.json" for i in range(len(missing_ids))],
    )
)

with multiprocessing.Pool() as pool:
    for res in tqdm(pool.starmap(sch_script.get_sch_paper, items)):
        results_missing.append(res)

#### Don't forget to add the new `results_missing` to data

### Final step to : clean and get data with vectors and citations

In [ ]:
data = data.drop_duplicates(subset=['arxiv_id']
data = data.dropna(subset=['sch_id'])
data = data.rename(columns={"arxiv_id": "id"})    

In [ ]:
merged_embeddings = pd.merge(emb, data, on='id', how='inner')
len(merged_embeddings)

In [ ]:
merged_embeddings.to_json('./final_redis_ds_data.json')